# Ray Tracing: Is our analytical approach going to work?

# Second question: At which time lags should we expect to see a peak in the auto / cross correlation signal?

To answer this question, we are going to compute the time difference between the direct P/S wave arrival, and the PPP and SH waves reflected off the mid-slab discontinuity, and the PPSSS and SH waves reflected off the Moho.

Load Python modules.

In [1]:
import numpy
from math import sqrt

Load functions from my own ray-tracing related modules.

In [2]:
from computeAngle import computeAngle3, computeAngle5
from computeTravelTime import computeTravelTime3, computeTravelTime5
from misc import latLon2xy

Get P- and S-wave velocities in the continental crust.

In [3]:
from data import VpCC, VsCC

Get latitudes, longitudes and names of the arrays.

In [4]:
from data import lat_a, lon_a, names_a, na

Get latitudes, longitudes and depths of the tremor sources.

In [5]:
from data import lat_s, lon_s, d_s, ns

Get latitude and longitude of the center of the (tremor sources) grid.

In [6]:
from data import lat0, lon0

Compute the distances from the arrays to the center of the (tremor sources) grid.

In [7]:
(x_a, y_a) = latLon2xy(lat_a, lon_a, lat0, lon0)

Compute the distances from the tremor sources to the center of the (tremor sources) grid

In [8]:
(x_s, y_s) = latLon2xy(lat_s, lon_s, lat0, lon0)

Initializations

In [9]:
tP = numpy.zeros((na, ns))
tS = numpy.zeros((na, ns))
tPPP = numpy.zeros((na, ns))
t3SH = numpy.zeros((na, ns))
tPPSSS = numpy.zeros((na, ns))
t5SH = numpy.zeros((na, ns))

Open output file

In [10]:
output = open('TimeDifferenceArray.txt', 'w')

Loop on arrays

In [11]:
for i in range(0, na):
    output.write('Array {}:\n'.format(names_a[i]))
    output.write('---------\n')
    # Loop on source position
    for j in range(0, ns):
        # Ray P
        tP[i, j] = sqrt((x_a[i] - x_s[j]) ** 2.0 + (y_a[i] - y_s[j]) ** 2.0 + d_s[j] ** 2.0) / VpCC
        # Ray S
        tS[i, j] = sqrt((x_a[i] - x_s[j]) ** 2.0 + (y_a[i] - y_s[j]) ** 2.0 + d_s[j] ** 2.0) / VsCC
        # Ray PPP
        angle = computeAngle3(x_s[j], y_s[j], d_s[j], x_a[i], y_a[i], 'P', 'P', 'P')
        tPPP[i, j] = computeTravelTime3(angle, x_s[j], y_s[j], d_s[j], x_a[i], y_a[i], 'P', 'P', 'P')
        # Ray 3SH
        angle = computeAngle3(x_s[j], y_s[j], d_s[j], x_a[i], y_a[i], 'S', 'S', 'S')
        t3SH[i, j] = computeTravelTime3(angle, x_s[j], y_s[j], d_s[j], x_a[i], y_a[i], 'S', 'S', 'S')
        # Ray PPSSS
        angle = computeAngle5(x_s[j], y_s[j], d_s[j], x_a[i], y_a[i], 'P', 'P', 'S', 'S', 'S')
        tPPSSS[i, j] = computeTravelTime5(angle, x_s[j], y_s[j], d_s[j], x_a[i], y_a[i], 'P', 'P', 'S', 'S', 'S')
        # Ray 5SH
        angle = computeAngle5(x_s[j], y_s[j], d_s[j], x_a[i], y_a[i], 'S', 'S', 'S', 'S', 'S')
        t5SH[i, j] = computeTravelTime5(angle, x_s[j], y_s[j], d_s[j], x_a[i], y_a[i], 'S', 'S', 'S', 'S', 'S')

    output.write('Autocorrelation - Horizontal component\n')
    output.write('\n')
    output.write('Time difference between S-wave and SH-wave (mid-slab)\n')
    output.write('Min value: {} s\n'.format(numpy.min(t3SH[i, :] - tS[i, :])))
    output.write('Max value: {} s\n'.format(numpy.max(t3SH[i, :] - tS[i, :])))
    output.write('Mean value: {} s\n'.format(numpy.mean(t3SH[i, :] - tS[i, :])))
    output.write('\n')
    output.write('Time difference between S-wave and SH-wave (Moho)\n')
    output.write('Min value: {} s\n'.format(numpy.min(t5SH[i, :] - tS[i, :])))
    output.write('Max value: {} s\n'.format(numpy.max(t5SH[i, :] - tS[i, :])))
    output.write('Mean value: {} s\n'.format(numpy.mean(t5SH[i, :] - tS[i, :])))
    output.write('\n')
    output.write('Time difference between S-wave and PPSSS-wave\n')
    output.write('Min value: {} s\n'.format(numpy.min(tPPSSS[i, :] - tS[i, :])))
    output.write('Max value: {} s\n'.format(numpy.max(tPPSSS[i, :] - tS[i, :])))
    output.write('Mean value: {} s\n'.format(numpy.mean(tPPSSS[i, :] - tS[i, :])))
    output.write('\n')

    output.write('Autocorrelation - Vertical component\n')
    output.write('\n')
    output.write('Time difference between P-wave and PPP-wave\n')
    output.write('Min value: {} s\n'.format(numpy.min(tPPP[i, :] - tP[i, :])))
    output.write('Max value: {} s\n'.format(numpy.max(tPPP[i, :] - tP[i, :])))
    output.write('Mean value: {} s\n'.format(numpy.mean(tPPP[i, :] - tP[i, :])))
    output.write('\n')

    output.write('Cross correlation - Horizontal and vertical component\n')
    output.write('\n')
    output.write('Time difference between S-wave and P-wave\n')
    output.write('Min value: {} s\n'.format(numpy.min(tS[i, :] - tP[i, :])))
    output.write('Max value: {} s\n'.format(numpy.max(tS[i, :] - tP[i, :])))
    output.write('Mean value: {} s\n'.format(numpy.mean(tS[i, :] - tP[i, :])))
    output.write('\n')   
    output.write('Time difference between S-wave and PPP-wave\n')
    output.write('Min value: {} s\n'.format(numpy.min(tS[i, :] - tPPP[i, :])))
    output.write('Max value: {} s\n'.format(numpy.max(tS[i, :] - tPPP[i, :])))
    output.write('Mean value: {} s\n'.format(numpy.mean(tS[i, :] - tPPP[i, :])))
    output.write('\n')
    output.write('Time difference between SH-wave (mid-slab) and P-wave\n')
    output.write('Min value: {} s\n'.format(numpy.min(t3SH[i, :] - tP[i, :])))
    output.write('Max value: {} s\n'.format(numpy.max(t3SH[i, :] - tP[i, :])))
    output.write('Mean value: {} s\n'.format(numpy.mean(t3SH[i, :] - tP[i, :])))
    output.write('\n')
    output.write('Time difference between PPSSS-wave and P-wave\n')
    output.write('Min value: {} s\n'.format(numpy.min(tPPSSS[i, :] - tP[i, :])))
    output.write('Max value: {} s\n'.format(numpy.max(tPPSSS[i, :] - tP[i, :])))
    output.write('Mean value: {} s\n'.format(numpy.mean(tPPSSS[i, :] - tP[i, :])))
    output.write('\n')   
    output.write('Time difference between SH-wave (Moho) and P-wave\n')
    output.write('Min value: {} s\n'.format(numpy.min(t5SH[i, :] - tP[i, :])))
    output.write('Max value: {} s\n'.format(numpy.max(t5SH[i, :] - tP[i, :])))
    output.write('Mean value: {} s\n'.format(numpy.mean(t5SH[i, :] - tP[i, :])))
    output.write('\n')

Average on all arrays.

In [12]:
output.write('All arrays:\n')
output.write('---------  \n')

output.write('Autocorrelation - Horizontal component\n')
output.write('\n')
output.write('Time difference between S-wave and SH-wave (mid-slab)\n')
output.write('Min value: {} s\n'.format(numpy.min(t3SH - tS)))
output.write('Max value: {} s\n'.format(numpy.max(t3SH - tS)))
output.write('Mean value: {} s\n'.format(numpy.mean(t3SH - tS)))
output.write('\n')
output.write('Time difference between S-wave and SH-wave (Moho)\n')
output.write('Min value: {} s\n'.format(numpy.min(t5SH - tS)))
output.write('Max value: {} s\n'.format(numpy.max(t5SH - tS)))
output.write('Mean value: {} s\n'.format(numpy.mean(t5SH - tS)))
output.write('\n')
output.write('Time difference between S-wave and PPSSS-wave\n')
output.write('Min value: {} s\n'.format(numpy.min(tPPSSS - tS)))
output.write('Max value: {} s\n'.format(numpy.max(tPPSSS - tS)))
output.write('Mean value: {} s\n'.format(numpy.mean(tPPSSS - tS)))
output.write('\n')

output.write('Autocorrelation - Vertical component\n')
output.write('\n')
output.write('Time difference between P-wave and PPP-wave\n')
output.write('Min value: {} s\n'.format(numpy.min(tPPP - tP)))
output.write('Max value: {} s\n'.format(numpy.max(tPPP - tP)))
output.write('Mean value: {} s\n'.format(numpy.mean(tPPP - tP)))
output.write('\n')

output.write('Cross correlation - Horizontal and vertical component\n')
output.write('\n')
output.write('Time difference between S-wave and P-wave\n')
output.write('Min value: {} s\n'.format(numpy.min(tS - tP)))
output.write('Max value: {} s\n'.format(numpy.max(tS - tP)))
output.write('Mean value: {} s\n'.format(numpy.mean(tS - tP)))
output.write('\n')   
output.write('Time difference between S-wave and PPP-wave\n')
output.write('Min value: {} s\n'.format(numpy.min(tS - tPPP)))
output.write('Max value: {} s\n'.format(numpy.max(tS - tPPP)))
output.write('Mean value: {} s\n'.format(numpy.mean(tS - tPPP)))
output.write('\n')
output.write('Time difference between SH-wave (mid-slab) and P-wave\n')
output.write('Min value: {} s\n'.format(numpy.min(t3SH - tP)))
output.write('Max value: {} s\n'.format(numpy.max(t3SH - tP)))
output.write('Mean value: {} s\n'.format(numpy.mean(t3SH - tP)))
output.write('\n')
output.write('Time difference between PPSSS-wave and P-wave\n')
output.write('Min value: {} s\n'.format(numpy.min(tPPSSS - tP)))
output.write('Max value: {} s\n'.format(numpy.max(tPPSSS - tP)))
output.write('Mean value: {} s\n'.format(numpy.mean(tPPSSS - tP)))
output.write('\n')   
output.write('Time difference between SH-wave (Moho) and P-wave\n')
output.write('Min value: {} s\n'.format(numpy.min(t5SH - tP)))
output.write('Max value: {} s\n'.format(numpy.max(t5SH - tP)))
output.write('Mean value: {} s\n'.format(numpy.mean(t5SH - tP)))
output.write('\n')

1

Close output file.

In [13]:
output.close()